In [ ]:
import Utils
class IntradayStraddleStrangle():
    def __init__(self, underlying, exposure , strike_round = 100):

        self.underlying = underlying
        self.exposure = exposure
        self.strike_round = strike_round
        self._ExecPrice =0

    @property
    def call_strike_method(self):
        pass
    @call_strike_method.setter
    def call_strike_method(self, method):
        self._call_strike_method = method
    @property
    def put_strike_method(self):
        pass
    @put_strike_method.setter
    def put_strike_method(self, method):
        self._put_strike_method = method
    @property
    def call_stoploss_method(self):
        pass
    @call_stoploss_method.setter
    def call_stoploss_method(self, method):
        self._call_stoploss_method = method
    @property
    def put_stoploss_method(self):
        pass
    @put_stoploss_method.setter
    def put_stoploss_method(self, method):
        self._put_stoploss_method = method
    @property
    def ExecPrice(self):
        return self._ExecPrice
    @ExecPrice.setter
    def ExecPrice(self, price : dict):
        self._ExecPrice = price
    @property
    def OrderSide(self):
        return self._OrderSide
    @OrderSide.setter
    def OrderSide(self, orderside):
        self._OrderSide = orderside
    @property
    def date_today(self):
        pass
    @date_today.setter
    def date_today(self, date):
        self._today = date

    @property
    def lot_size(self):
        return self._lot_size
    @lot_size.setter
    def lot_size(self, lot_size):
        self._lot_size = lot_size
    @property
    def spot_signal_price(self):
        pass
    @spot_signal_price.setter
    def spot_signal_price (self, spot_value):
        self._spot_signal_price = spot_value

    def get_entry_stikes(self):

       self.ATM = round(self._spot_signal_price /self.strike_round,0) *self.strike_round
       call_strike = (self._call_strike_method(spot_signal_price = self._spot_signal_price, ATM = self.ATM))
       put_strike  = (self._put_strike_method(spot_signal_price = self._spot_signal_price, ATM = self.ATM))

       return {'Underlying' : self.underlying, 'ATM' : self.ATM, 'ExpiryDate' :Utils.currentweekexpiry(self._today), 'Call_Strike': call_strike, 'Put_Strike' : put_strike}

    def get_entry_trades(self):
        call_trade_dict = {}
        put_trade_dict  = {}

        strikes = self.get_entry_stikes()
        ATM = strikes['ATM']
        Expiry = strikes['ExpiryDate']
        call_strike = strikes['Call_Strike']
        put_strike  = strikes['Put_Strike']

        #Call
        call_trade_dict['Underlying'] = self.underlying
        call_trade_dict['OptionType'] = 'CE'
        call_trade_dict['Strike_Pr']  = call_strike
        call_trade_dict['ExpiryDate'] = Expiry
        call_trade_dict['OrderSide']  = self.OrderSide
        call_trade_dict['OrderType']  = 'Market'
        call_trade_dict['StopPrice']  = 0
        call_trade_dict['OrderQuantity'] = round(self.exposure/ATM/self.lot_size,0) * self.lot_size

        #Put
        put_trade_dict['Underlying'] = self.underlying
        put_trade_dict['OptionType'] = 'PE'
        put_trade_dict['Strike_Pr']  = put_strike
        put_trade_dict['ExpiryDate'] = Expiry
        put_trade_dict['OrderSide']  = self.OrderSide
        put_trade_dict['OrderType']  = 'Market'
        put_trade_dict['StopPrice']  =  0
        put_trade_dict['OrderQuantity'] = round(self.exposure/ATM/self.lot_size,0) * self.lot_size

        self._trades =  {'Call_trades': call_trade_dict,
                         'Put_trades' : put_trade_dict}
        return self._trades

    def get_exit_trade(self, call_execution_price =-1 ,put_execution_price =-1 ):
        call_trade_dict = {}
        put_trade_dict  = {}

        #Call order
        if call_execution_price !=-1 :
            call_trade_dict['Underlying'] = self.underlying
            call_trade_dict['OptionType'] = 'CE'
            call_trade_dict['Strike_Pr']  = self._trades['Call_trades']['Strike_Pr']
            call_trade_dict['ExpiryDate'] = self._trades['Call_trades']['ExpiryDate']
            call_trade_dict['OrderSide']  = Utils.oppositetrade(self._trades['Call_trades']['OrderSide'])
            call_trade_dict['OrderType']  = 'StopMarket'
            call_trade_dict['StopPrice']  = Utils.roundtoNSEtick(self._call_stoploss_method(spot_signal_price = self._spot_signal_price, ATM = self.ATM, ExecPrice = self.ExecPrice['call']))
            call_trade_dict['OrderQuantity'] = self._trades['Call_trades']['OrderQuantity']

        #Put order
        if put_execution_price !=-1 :
            put_trade_dict['Underlying'] = self.underlying
            put_trade_dict['OptionType'] = 'PE'
            put_trade_dict['Strike_Pr']  = self._trades['Put_trades']['Strike_Pr']
            put_trade_dict['ExpiryDate'] = self._trades['Put_trades']['ExpiryDate']
            put_trade_dict['OrderSide']  = Utils.oppositetrade(self._trades['Put_trades']['OrderSide'])
            put_trade_dict['OrderType']  = 'StopMarket'
            put_trade_dict['StopPrice']  = Utils.roundtoNSEtick(self._put_stoploss_method(spot_signal_price = self._spot_signal_price, ATM = self.ATM, ExecPrice = self.ExecPrice['put']))
            put_trade_dict['OrderQuantity'] = self._trades['Put_trades']['OrderQuantity']
        return {'Call_trades': call_trade_dict, 'Put_trades' : put_trade_dict}

class IntradayStrikeMethods():
    def constantoffset(self,spot_signal_price, ATM ,offset = 200):
        return ATM + offset
    def pct_offset_from_spot(self, spot_signal_price, ATM, pct = 1.5, strike_round=100):
        return round(spot_signal_price * (1 +pct/100) /100 , 0)*100

class IntradayStoplossMethods():
    def fixed_pct_stoploss_on_premium(self,spot_signal_price, ATM, ExecPrice, pct = 20):
        #print(spot_signal_price, ATM, ExecPrice)
        return ExecPrice * (1+pct/100)
    def max_of_spot_and_premium(self, spot_signal_price, ATM, ExecPrice, spot_pct=0.2, premium_pct = 20 ):
        #print(spot_signal_price, ATM, ExecPrice)
        return max(ExecPrice * premium_pct/100 , spot_signal_price * spot_pct/100) + ExecPrice


In [ ]:
# Logic
# Nifty Intra-day options strategy
# Daily Short straddle on Nifty ATM options with 50% stop loss on each leg and stop loss to cost for other leg if stop loss on any one leg gets triggered.
# Targeted returns is +50% on capital allocated of INR 1.40 lacs margin per lot.

# Strategy:
# 1)	Sell ATM+100 put and ATM-100 call at 9:19 am for all days of the week.
# 2)	Close open positions at 3:26 pm if stop loss and/or stop loss at cost trades not triggered.
# 3)	Stop loss at +50% on entry price of individual leg (if entry price is 100 then SL will be 150)
# 4)	If stop loss gets hit on one Leg, then move stop loss of other leg to cost.

class nifty_strategy_define():

    def __init__(self, underlying, exposure, strike_round = 100):
        self.Straddle = basic_framework.IntradayStraddleStrangle(underlying, exposure, strike_round)

    def _specificvariations(self):
        intraentrytrade='SELL'
        intraoffset=100
        intrastoploss=1.5
        return (intraoffset, intraentrytrade, intrastoploss)

    @property
    def date_today(self):
        pass
    @date_today.setter
    def date_today(self, date):
        self._today = date
        self.Straddle.date_today = self._today

        #set strike methods
        (intraoffset, intraentrytrade, intrastoploss) = self._specificvariations()
        self.Straddle.call_strike_method = lambda spot_signal_price, ATM : round(self.Straddle._spot_signal_price/self.Straddle.strike_round,0)*100 - intraoffset
        self.Straddle.put_strike_method = lambda spot_signal_price, ATM : round(self.Straddle._spot_signal_price/self.Straddle.strike_round,0)*100 + intraoffset
        self.Straddle.put_stoploss_method = lambda spot_signal_price, ATM ,ExecPrice : ExecPrice * intrastoploss
        self.Straddle.call_stoploss_method = lambda spot_signal_price, ATM ,ExecPrice : ExecPrice * intrastoploss
        self.Straddle.OrderSide = intraentrytrade

    def modify_stoploss_to_execution_price(self, modify_pe_stoploss = False, modify_ce_stoploss = False):
        if modify_ce_stoploss :
            self.Straddle.call_stoploss_method = lambda spot_signal_price, ATM ,ExecPrice : ExecPrice * 1
        elif modify_pe_stoploss:
            self.Straddle.put_stoploss_method = lambda spot_signal_price, ATM ,ExecPrice : ExecPrice * 1

In [ ]:
import AutoChooseBidAsk_MinData,GDFL_1MinData
import Utils
import datetime as dt

d = GDFL_1MinData.GDFL_1MIN_OHLC()

expiry_parser = lambda expiry_str : dt.datetime.strptime(expiry_str , '%d%b%y')
DR = AutoChooseBidAsk_MinData()

parametersNifty = {'underlying' : 'NIFTY', 'IndexSymbol' : 'NIFTY 50', 'lOT SIZE' : 50 , 'OPTION_START_DATE': dt.date(2019,2,1),'Strike_offset' : 200}
underlyingParameters =  parametersNifty

#Initialise Strategy
IntraStrat = nifty_strategy_define(underlying =underlyingParameters['underlying'], exposure = 10**7, strike_round = 100)

IntraStrat.Straddle.lot_size = underlyingParameters['lOT SIZE']

#Backtest Loop
backtest_start_date = dt.date(2019,3,1)
backtest_end_date = dt.date(2022,6,23)
tempdays = [backtest_start_date +dt.timedelta(days=x) for x in range((backtest_end_date-backtest_start_date).days+1)]
backtestdates = [x for x in tempdays if x not in Utils.holidays and x.weekday() not in [5,6]]
print(len(backtestdates))

all_call_trades = list()
all_put_trades  = list()

for trade_date in backtestdates:
    try :
        # 09:19 index ltp
        index_bid_ask_data = DR.read_indices_data(underlyingParameters['IndexSymbol'], trade_date)
        time0919 = dt.datetime.combine(trade_date, dt.time(9,18))
        LTP0919 = SearchExecutionPrice(index_bid_ask_data).auto_search.get_ltp(time0919)

        IntraStrat.date_today = trade_date
        IntraStrat.Straddle.spot_signal_price = LTP0919
        entryIns = IntraStrat.Straddle.get_entry_stikes()

        #entry data
        call_bid_ask_df = DR.read_options_data(entryIns['Underlying'], int(entryIns['Call_Strike']), 'CE', trade_date, expiry_parser(entryIns['ExpiryDate']))
        put_bid_ask_df  =DR.read_options_data(entryIns['Underlying'], int(entryIns['Put_Strike']), 'PE', trade_date, expiry_parser(entryIns['ExpiryDate']))
        entryTrades = IntraStrat.Straddle.get_entry_trades()
        time1500 = dt.datetime.combine(trade_date, dt.time(15,26))
        call_trades = entryTrades['Call_trades']
        put_trades  = entryTrades['Put_trades']
        call_trade_exec = SearchExecutionPrice(call_bid_ask_df).auto_search.market_order_execution_price(timestamp= time0919, order_side = call_trades['OrderSide'])
        put_trade_exec = SearchExecutionPrice(put_bid_ask_df).auto_search.market_order_execution_price(timestamp=time0919, order_side = put_trades['OrderSide'])

        #check Price
        call_execution_price = call_trade_exec
        put_execution_price  = put_trade_exec

        IntraStrat.Straddle.ExecPrice= {'call': call_execution_price, 'put': put_execution_price}

        #Exit after Stoploss
        time1526 = dt.datetime.combine(trade_date, dt.time(15,26))
        if call_execution_price !=-1 or put_execution_price !=-1 :
            exitTrades = IntraStrat.Straddle.get_exit_trade(call_execution_price, put_execution_price)

        if call_execution_price !=-1 :
            #Call trades
            entry_call_trades = call_trades.copy()
            entry_call_trades['ExecPrice']  = call_execution_price
            entry_call_trades['Timestamp']  = time0919
            all_call_trades.append(entry_call_trades)
            exit_call_trades = exitTrades['Call_trades']

            call_trade_exit_exec = SearchExecutionPrice(call_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_call_trades['StopPrice'], order_starttime = time0919, order_endtime = time1526, order_side = exit_call_trades['OrderSide'])
        if put_execution_price !=-1 :
            entry_put_trades = put_trades.copy()
            entry_put_trades['ExecPrice']  = put_execution_price
            entry_put_trades['Timestamp']  = time0919
            all_put_trades.append(entry_put_trades)
            exit_put_trades  = exitTrades['Put_trades']

            put_trade_exit_exec = SearchExecutionPrice(put_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_put_trades['StopPrice'], order_starttime = time0919, order_endtime = time1526, order_side = exit_put_trades['OrderSide'])
        #Check Stoploss Hit
        if call_trade_exit_exec[0] ==True and put_trade_exit_exec[0] ==True:

            if call_trade_exit_exec[2] > put_trade_exit_exec[2] :  #modify call stoploss
                IntraStrat.modify_stoploss_to_execution_price(modify_ce_stoploss = True)
                exitTrades = IntraStrat.Straddle.get_exit_trade(call_execution_price, put_execution_price)
                exit_call_trades = exitTrades['Call_trades']
                call_trade_exit_exec = SearchExecutionPrice(call_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_call_trades['StopPrice'], order_starttime = put_trade_exit_exec[2], order_endtime = time1526, order_side = exit_call_trades['OrderSide'])

            else : #modify put trades

                IntraStrat.modify_stoploss_to_execution_price(modify_pe_stoploss = True)
                exitTrades = IntraStrat.Straddle.get_exit_trade(call_execution_price, put_execution_price)
                exit_put_trades = exitTrades['Put_trades']
                put_trade_exit_exec = SearchExecutionPrice(put_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_put_trades['StopPrice'], order_starttime = call_trade_exit_exec[2], order_endtime = time1526, order_side = exit_put_trades['OrderSide'])

        elif put_trade_exit_exec[0] ==True :    #modify call trades
            IntraStrat.modify_stoploss_to_execution_price(modify_ce_stoploss = True)
            exitTrades = IntraStrat.Straddle.get_exit_trade(call_execution_price, put_execution_price)
            exit_call_trades = exitTrades['Call_trades']
            call_trade_exit_exec = SearchExecutionPrice(call_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_call_trades['StopPrice'], order_starttime = put_trade_exit_exec[2], order_endtime = time1526, order_side = exit_call_trades['OrderSide'])

        elif call_trade_exit_exec[0] ==True :    #modify put trades

            IntraStrat.modify_stoploss_to_execution_price(modify_pe_stoploss = True)
            exitTrades = IntraStrat.Straddle.get_exit_trade(call_execution_price, put_execution_price)
            exit_put_trades = exitTrades['Put_trades']
            put_trade_exit_exec = SearchExecutionPrice(put_bid_ask_df).auto_search.stop_market_execution_price(stop_price = exit_put_trades['StopPrice'], order_starttime = call_trade_exit_exec[2], order_endtime = time1526,order_side = exit_put_trades['OrderSide'] )

        #exit
        if call_execution_price !=-1:
            if call_trade_exit_exec[0] ==False :
                call_time_exit_price = SearchExecutionPrice(call_bid_ask_df).auto_search.market_order_execution_price(time1526,order_side = exit_call_trades['OrderSide'])

                Exit_call_trades = exit_call_trades.copy()
                Exit_call_trades['ExecPrice'] = call_time_exit_price
                Exit_call_trades['Timestamp'] = time1526
                all_call_trades.append(Exit_call_trades)

            else:
                Exit_call_trades = exit_call_trades.copy()
                Exit_call_trades['ExecPrice'] = call_trade_exit_exec[1]
                Exit_call_trades['Timestamp'] = call_trade_exit_exec[2]
                all_call_trades.append(Exit_call_trades)

        if put_execution_price !=-1:
            if put_trade_exit_exec[0] ==False :
                put_time_exit_price = SearchExecutionPrice(put_bid_ask_df).auto_search.market_order_execution_price(time1526,order_side = exit_put_trades['OrderSide'])

                Exit_put_trades = exit_put_trades.copy()
                Exit_put_trades['ExecPrice'] = put_time_exit_price
                Exit_put_trades['Timestamp'] = time1526
                all_put_trades.append(Exit_put_trades)

            else:
                Exit_put_trades = exit_put_trades.copy()
                Exit_put_trades['ExecPrice'] = put_trade_exit_exec[1]
                Exit_put_trades['Timestamp'] = put_trade_exit_exec[2]
                all_put_trades.append(Exit_put_trades)

        print(trade_date)

import pandas as pd
pd.DataFrame(all_call_trades).to_csv('./NiftyATM100_call.csv')
pd.DataFrame(all_put_trades).to_csv('./NiftyATM100_put.csv')